In [1]:
# ================================================
# ✅ 1️⃣ LIBRARIES & SETUP
# ================================================
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import torchvision.transforms as transforms
import torchvision.models as models

# ================================================
# ✅ 2️⃣ PATHS
# ================================================
image_dir = "/kaggle/input/basem/images"
input_csv = "/kaggle/input/basem/dataset.csv"

# ================================================
# ✅ 3️⃣ LOAD & PREPROCESS CSV
# ================================================
df = pd.read_csv(input_csv)

existing_data = []
for _, row in df.iterrows():
    image_filename = row['image_path']
    full_image_path = os.path.join(image_dir, image_filename)
    if os.path.exists(full_image_path):
        label_converted = row['label 2'] - 1
        existing_data.append({
            'Image_path': full_image_path,
            'Label_Sentiment': label_converted
        })

processed_df = pd.DataFrame(existing_data)

# ================================================
# ✅ 4️⃣ TRAIN/VAL/TEST SPLIT
# ================================================
train_df, temp_df = train_test_split(processed_df, test_size=0.3, stratify=processed_df['Label_Sentiment'], random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=1/3, stratify=temp_df['Label_Sentiment'], random_state=42)

for df_name, df_ in [('train', train_df), ('test', test_df), ('val', val_df)]:
    df_['label'] = df_['Label_Sentiment']
    df_.to_csv(f'/kaggle/working/{df_name}_vision_only.csv', index=False)

print(f"Train samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")

# ================================================
# ✅ 5️⃣ DEVICE SETUP
# ================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ================================================
# ✅ 6️⃣ IMAGE TRANSFORMS
# ================================================
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ================================================
# ✅ 7️⃣ DATASET CLASS
# ================================================
class VisionOnlyDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['Image_path']
        label = row['label']
        
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            # Create a black image as fallback
            image = Image.new('RGB', (224, 224), color='black')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# ================================================
# ✅ 8️⃣ DATALOADERS
# ================================================
batch_size = 16

train_dataset = VisionOnlyDataset(train_df, transform=train_transform)
val_dataset = VisionOnlyDataset(val_df, transform=val_test_transform)
test_dataset = VisionOnlyDataset(test_df, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# ================================================
# ✅ 9️⃣ RESNET50 MODEL
# ================================================
class ResNet50Classifier(nn.Module):
    def __init__(self, num_classes=3, dropout=0.5):
        super(ResNet50Classifier, self).__init__()
        # Load pre-trained ResNet50
        self.resnet50 = models.resnet50(pretrained=True)
        
        # Freeze early layers (optional - uncomment to freeze)
        # for param in list(self.resnet50.parameters())[:-20]:
        #     param.requires_grad = False
        
        # Get the number of features from the last layer
        num_features = self.resnet50.fc.in_features
        
        # Replace the classifier
        self.resnet50.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.resnet50(x)

# ================================================
# ✅ 🔟 MODEL INITIALIZATION
# ================================================
model = ResNet50Classifier(num_classes=3, dropout=0.5).to(device)

# Print model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# ================================================
# ✅ 1️⃣1️⃣ LOSS & OPTIMIZER
# ================================================
# Calculate class weights for balanced training
class_weights = train_df['label'].value_counts().sort_index().tolist()
total = sum(class_weights)
weights = [total / c for c in class_weights]
print(f"Class distribution: {class_weights}")
print(f"Class weights: {weights}")

criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(weights).to(device))
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# ================================================
# ✅ 1️⃣2️⃣ TRAINING LOOP
# ================================================
num_epochs = 25
patience = 5
patience_counter = 0
best_val_loss = float('inf')
best_val_acc = 0.0

train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    # ============================================================
    # TRAINING PHASE
    # ============================================================
    model.train()
    total_train_loss = 0
    train_correct = 0
    train_total = 0

    for images, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        
        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = train_correct / train_total
    
    # ============================================================
    # VALIDATION PHASE
    # ============================================================
    model.eval()
    total_val_loss = 0
    val_predictions = []
    val_labels_list = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_val_loss += loss.item()
            
            # Store predictions for metrics
            _, predicted = torch.max(outputs.data, 1)
            val_predictions.extend(predicted.cpu().numpy())
            val_labels_list.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_labels_list, val_predictions)
    
    # Store metrics
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)
    
    # Update learning rate
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")
    print(f"Learning Rate: {current_lr:.6f}")
    print("-" * 50)

    # ============================================================
    # EARLY STOPPING CHECK
    # ============================================================
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_val_acc = val_accuracy
        patience_counter = 0
        torch.save(model.state_dict(), "best_resnet50_model.pt")
        print("✅ Validation loss improved — model saved.")
    else:
        patience_counter += 1
        print(f"⏰ No improvement — patience {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break
    
    print()

# ================================================
# ✅ 1️⃣3️⃣ FINAL TEST EVALUATION
# ================================================
print("\n🔍 Loading best model for final evaluation...")
model.load_state_dict(torch.load("best_resnet50_model.pt"))
model.eval()

test_predictions = []
test_labels_list = []
total_test_loss = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Final Test Evaluation"):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        total_test_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        test_predictions.extend(predicted.cpu().numpy())
        test_labels_list.extend(labels.cpu().numpy())

# Calculate final metrics
test_accuracy = accuracy_score(test_labels_list, test_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels_list, test_predictions, average='weighted')
cm = confusion_matrix(test_labels_list, test_predictions)

print("\n" + "="*60)
print("📊 FINAL TEST RESULTS - VISION ONLY (ResNet50)")
print("="*60)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test Loss: {total_test_loss/len(test_loader):.4f}")
print(f"\nBest Validation Accuracy: {best_val_acc:.4f}")
print(f"Best Validation Loss: {best_val_loss:.4f}")

print(f"\nConfusion Matrix:")
print("Predicted ->")
print("   0   1   2")
for i, row in enumerate(cm):
    print(f"{i}: {row}")

# Class-wise metrics
precision_class, recall_class, f1_class, support = precision_recall_fscore_support(test_labels_list, test_predictions, average=None)
print(f"\n📈 Class-wise Metrics:")
for i in range(len(precision_class)):
    print(f"Class {i}: Precision={precision_class[i]:.4f}, Recall={recall_class[i]:.4f}, F1={f1_class[i]:.4f}, Support={support[i]}")

print("\n🎯 Summary:")
print(f"Vision-only model (ResNet50) achieved {test_accuracy:.4f} accuracy on test set")
print("="*60)

Train samples: 3156
Validation samples: 451
Test samples: 902
Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 199MB/s]


Total parameters: 24,558,659
Trainable parameters: 24,558,659
Class distribution: [1404, 1237, 515]
Class weights: [2.247863247863248, 2.551333872271625, 6.128155339805825]


Validation Epoch 1: 100%|██████████| 29/29 [00:08<00:00,  3.31it/s]


Epoch [1/25]
Train Loss: 0.9921 | Train Acc: 0.5086
Val Loss: 0.9573 | Val Acc: 0.5632
Learning Rate: 0.000100
--------------------------------------------------
✅ Validation loss improved — model saved.



...

Epoch [8/25]
Train Loss: 0.6051 | Train Acc: 0.7357
Val Loss: 1.1123 | Val Acc: 0.5876
Learning Rate: 0.000050
--------------------------------------------------
⏰ No improvement — patience 4/5



Validation Epoch 9: 100%|██████████| 29/29 [00:06<00:00,  4.77it/s]


Epoch [9/25]
Train Loss: 0.5727 | Train Acc: 0.7433
Val Loss: 1.1651 | Val Acc: 0.6164
Learning Rate: 0.000050
--------------------------------------------------
⏰ No improvement — patience 5/5
🛑 Early stopping triggered at epoch 9

🔍 Loading best model for final evaluation...


Final Test Evaluation: 100%|██████████| 57/57 [00:15<00:00,  3.57it/s]


📊 FINAL TEST RESULTS - VISION ONLY (ResNet50)
Test Accuracy: 0.6020
Test Precision: 0.6700
Test Recall: 0.6020
Test F1-Score: 0.6131
Test Loss: 0.8156

Best Validation Accuracy: 0.6009
Best Validation Loss: 0.8641

Confusion Matrix:
Predicted ->
   0   1   2
0: [216 124  62]
1: [ 30 227  96]
2: [  8  39 100]

📈 Class-wise Metrics:
Class 0: Precision=0.8504, Recall=0.5373, F1=0.6585, Support=402
Class 1: Precision=0.5821, Recall=0.6431, F1=0.6110, Support=353
Class 2: Precision=0.3876, Recall=0.6803, F1=0.4938, Support=147

🎯 Summary:
Vision-only model (ResNet50) achieved 0.6020 accuracy on test set
